In [1]:
import pandas as pd
from xbbg import blp

### Get Tickers

In [2]:
index_name = 'YCGT0025 Index'
TICKS_BENCH = ['CT02 Govt','CT05 Govt','CT07 Govt','CT10 Govt','CT20 Govt','CT30 Govt']
TICKS_FUT = [
    'TY',
    'FV',
    'TU',
    '3Y',
    'TWE',
    'WN',
    'US',
    'UXY',
    ]
TICKS_FUT = [f'{x}A Comdty' for x in TICKS_FUT]

FLD_VOLUME = 'PX_VOLUME'
FLD_VWAP = 'VWAP_VOLUME'
FLDS = ['SECURITY_NAME','ISSUE_DT','PX_LAST','PX_DISC_MID','DUR_MID','TREASURY_BILL_INDICATOR','FUT_CONT_SIZE','FUT_SEC_SPEC_ML']

treasury_members = blp.bds(index_name, 'INDX_MEMBERS')

### Info and VWAP

In [3]:
TICKS_ACTIVE_CURVE = treasury_members['member_ticker_and_exchange_code'].tolist()
TICK_LQA = 'LQA_EXPECTED_DAILY_VOLUME'
TICK_LQA_ALT = 'LQA_EMPIRICAL_TOTAL_TRADE_VOLUME'

tickers = TICKS_ACTIVE_CURVE.copy()
tickers += TICKS_BENCH
tickers += TICKS_FUT

volume = blp.bdp(tickers,flds=FLD_VOLUME)
vwap = blp.bdp(tickers,flds=FLD_VWAP)
info = blp.bdp(tickers,flds=FLDS)
lqa = blp.bdp(tickers,flds=TICK_LQA)
lqa_alt = blp.bdp(tickers,flds=TICK_LQA_ALT)

### Organize Data

In [4]:
df = pd.concat([info,volume,vwap,lqa,lqa_alt],axis=1)
df.rename(columns={TICK_LQA_ALT.lower():'lqa alt','fut_sec_spec_ml':'margin','fut_cont_size':'contract size','px_disc_mid':'px disc','treasury_bill_indicator':'is tbill','dur_mid':'duration','pv01_best':'pv01','px_volume':'px volume','vwap_turnover':'volume','vwap_volume':'vwap','issue_dt':'issue date','security_name':'name','px_last':'px last',TICK_LQA.lower():'lqa'}, inplace=True)

In [5]:
mask = df['is tbill'] == 'Y'
df.loc[mask,'price'] = df.loc[mask,'px disc']
df.loc[~mask,'price'] = df.loc[~mask,'px last']

In [6]:
df = df.drop_duplicates(subset='name', keep='first')
df_orig = df.copy()

In [7]:
# Set to true to use mkt val instead of face value
DO_MKT_VAL = True
# Set to true to use BB LQA for futures volume, false to use actual volume
USE_LQA_FOR_FUTURES = True
# Assume LQA value for cash bonds is per $100 face
USE_LQA_CASH_PER_FACE = True
# Use LQA which needs scaled by days per month
USE_LQA_ALT = False

if USE_LQA_ALT:
    df.loc[TICKS_ACTIVE_CURVE,'lqa'] = df.loc[TICKS_ACTIVE_CURVE,'lqa alt'] / 21.0  # Assuming 21 trading days in a month


FACE = 100

df.loc[TICKS_ACTIVE_CURVE,'volume notional'] = df.loc[TICKS_ACTIVE_CURVE,'lqa']
if USE_LQA_CASH_PER_FACE:
    df.loc[TICKS_ACTIVE_CURVE,'volume notional'] *= FACE

if USE_LQA_FOR_FUTURES:
    df.loc[TICKS_FUT,'volume notional'] = df.loc[TICKS_FUT,'lqa']
else:
    df.loc[TICKS_FUT,'volume notional'] = df.loc[TICKS_FUT,'px volume']

df.loc[TICKS_FUT,'volume notional'] *= df.loc[TICKS_FUT,'contract size']

if DO_MKT_VAL:
    df.loc[TICKS_ACTIVE_CURVE,'volume notional'] *= df.loc[TICKS_ACTIVE_CURVE,'price'] / FACE
    df.loc[TICKS_FUT,'volume notional'] *= df.loc[TICKS_FUT,'price']  / FACE

df.loc[TICKS_ACTIVE_CURVE,'volume dv01'] = df.loc[TICKS_ACTIVE_CURVE,'volume notional'] * df.loc[TICKS_ACTIVE_CURVE,'duration'] * (1/100/100)
df.loc[TICKS_FUT,'volume dv01'] = df.loc[TICKS_FUT,'volume notional'] * df.loc[TICKS_FUT,'duration'] * (1/100/100)


df.loc[TICKS_FUT,'margin fraction'] = df.loc[TICKS_FUT,'margin'] / (df.loc[TICKS_FUT,'contract size'] * df.loc[TICKS_FUT,'price'] / FACE)
df.loc[TICKS_ACTIVE_CURVE,'volume cash'] = df.loc[TICKS_ACTIVE_CURVE,'volume notional'].copy()
df.loc[TICKS_FUT,'volume cash'] = df.loc[TICKS_FUT,'volume notional'] * df.loc[TICKS_FUT,'margin fraction']

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_26192/1936874373.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[189905634860.7575 186890065195.94876 224149898999.10162
 1103831547.4567904 nan 37414037286.7295 43336350678.637405
 71022745756.52098]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[TICKS_FUT,'volume notional'] *= df.loc[TICKS_FUT,'contract size']


***

In [8]:
OUTPUT_FILE = '../data/treasury_volume_metrics.xlsx'

with pd.ExcelWriter(OUTPUT_FILE) as writer:
    df[['name','issue date','price','volume notional', 'duration', 'volume dv01', 'margin fraction', 'volume cash']].to_excel(writer, sheet_name='treasury trading metrics',index=True)
    df.to_excel(writer, sheet_name='treasury trading data',index=True)
print(f"Exported metrics to {OUTPUT_FILE}")


Exported metrics to ../data/treasury_volume_metrics.xlsx


***

In [9]:
format_dict = {
    **dict.fromkeys(['volume cash', 'volume notional', 'volume dv01', 'lqa', 'lqa alt','vwap'], '{:.0e}'),
    'margin fraction': '{:.0%}',
    **dict.fromkeys(['price', 'duration', 'px last', 'px disc'], '{:.2f}'),
    **dict.fromkeys(['margin', 'contract size', 'px volume'], '{:,.0f}')
}

# apply all at once
styled = df.style.format(format_dict, na_rep='')
display(styled)

,name,issue date,px last,px disc,duration,is tbill,contract size,margin,px volume,vwap,lqa,lqa alt,price,volume notional,volume dv01,margin fraction,volume cash
3YA Comdty,US 3YR NOTE (CBT) Sep25,,106.27,106.27,2.85,,"200,000","1,800",252,1e+02,6e+03,,106.27,1e+09,3e+05,1%,1e+07
912797MG Govt,B 08/07/25,2024-08-08,4.22,99.58,0.10,Y,,,,5e+12,3e+08,3e+09,99.58,3e+10,3e+05,,3e+10
912797MS Govt,B 10/02/25,2024-10-03,4.19,98.92,0.25,Y,,,,5e+12,4e+08,4e+08,98.92,3e+10,9e+05,,3e+10
912797QC Govt,B 07/29/25,2025-04-01,4.13,99.69,0.07,Y,,,,5e+12,5e+08,1e+09,99.69,5e+10,4e+05,,5e+10
912797QL Govt,B 08/26/25,2025-04-29,4.30,99.34,0.15,Y,,,,5e+12,4e+08,6e+08,99.34,4e+10,6e+05,,4e+10
912797QX Govt,B 06/11/26,2025-06-12,3.79,96.37,0.96,Y,,,,5e+12,1e+08,3e+08,96.37,1e+10,1e+06,,1e+10
912797RA Govt,B 01/02/26,2025-07-03,4.08,97.91,0.50,Y,,,,4e+12,,,97.91,,,,
912797RE Govt,B 10/28/25,2025-07-01,4.18,98.62,0.32,Y,,,,5e+12,1e+08,7e+10,98.62,9e+09,3e+05,,9e+09
912810UK Govt,T 4 3/4 05/15/55,2025-05-15,99.95,99.94,16.15,,,,,1e+13,2e+08,1e+09,99.95,2e+10,3e+07,,2e+10
912810UL Govt,T 5 05/15/45,2025-06-02,103.17,103.16,12.86,,,,,9e+12,1e+08,2e+08,103.17,1e+10,1e+07,,1e+10
